## Import packages

In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

## Data Preperation

In [2]:
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape
print(n_samples, n_features)

569 30


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=12 )

In [4]:
def convert_to_tensor(*numpy_arrays):
    """
    :param numpy_arrays: list of numpy arrays
    :return tensors: list of torch.tensor
    """
    return [torch.from_numpy(numpy_array.astype(np.float32)) for numpy_array in numpy_arrays]

In [5]:
# scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [6]:
X_train, X_test, y_train, y_test = convert_to_tensor(X_train, X_test, y_train, y_test)

In [7]:
y_train = y_train.view(y_train.shape[0], -1)
y_test = y_test.view(y_test.shape[0], -1)

In [8]:
class LogisticRegression(nn.Module):

    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        return torch.sigmoid(self.linear(x))

model = LogisticRegression(n_features)


In [9]:
# 2) loss and optimizer
learning_rate = .01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)



In [10]:
# 3) training loop
n_epochs = 100

for epoch in range(n_epochs):

    # forward
    y_predicted = model(X_train)
    loss = criterion(y_predicted, y_train)

    # backward
    loss.backward()

    # updates
    optimizer.step()

    optimizer.zero_grad()

    if (epoch + 1) % 10 == 0:
        print(f"epoch: {epoch + 1}, loss = {loss.item():.4f}")

with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / y_test.shape[0]
    print(f"Accuracy {acc:.4f}")





epoch: 10, loss = 0.5856
epoch: 20, loss = 0.4838
epoch: 30, loss = 0.4178
epoch: 40, loss = 0.3715
epoch: 50, loss = 0.3370
epoch: 60, loss = 0.3103
epoch: 70, loss = 0.2889
epoch: 80, loss = 0.2713
epoch: 90, loss = 0.2565
epoch: 100, loss = 0.2439
Accuracy 0.9441
